In [1]:
import pandas as pd
import joblib
import os

In [2]:
data = pd.read_parquet('Data/test_all_slopes.parquet')
data.shape

(924621, 1280)

In [3]:
def predict(data):
    
    models = [
        'fold_0_iter',
        'fold_1_iter',
        'fold_2_iter',
        'fold_3_iter',
        'fold_4_iter',
    ]

    model_list = []
    for path in ['models_DART_slope/']:
        for fname in os.listdir(path):
            for model_name in models:
                if model_name in fname:
                    model_list.append(path + fname)

    pred_list = []
    for model_path in model_list:
        print(model_path)
        print('{:.02f} MB'.format(os.path.getsize(model_path)/1000000))
        model = joblib.load(model_path)
        pred_list.append(model.predict(data))
    
    return pred_list

In [4]:
pred_list = predict(data)

models_DART_slope/fold_4_iter_10204_score_0.79878.pkl
117.69 MB
models_DART_slope/fold_1_iter_7564_score_0.79535.pkl
87.39 MB
models_DART_slope/fold_2_iter_9796_score_0.79837.pkl
113.09 MB
models_DART_slope/fold_0_iter_8170_score_0.80355.pkl
94.27 MB
models_DART_slope/fold_3_iter_9478_score_0.79343.pkl
109.41 MB


In [5]:
column_name = [
    'fold_4',
    'fold_1',
    'fold_2',
    'fold_0',
    'fold_3',
]

pred_df = pd.DataFrame(pred_list).T
pred_df.columns = column_name
pred_df.index = data.index

In [6]:
pred_df.mean(axis=1).to_csv('Output/p_LgbmDART_AggPcaDiffRoundSlope_HT1_1500_L1.csv', header=['prediction'])

In [ ]:
p1 = pred_df.mean(axis=1)
p2 = pd.read_csv('Output/p_blend_kaggle.csv', index_col='customer_ID').loc[pred_df.index]

In [ ]:
p_df = pd.concat([p1, p2], axis=1)
p_df

In [ ]:
ensemble = 0.05*p_df[0] + 0.95*p_df['prediction'] 

In [ ]:
ensemble.to_csv('Output/p_ensemble_LgbmDART_AggPcaDiffRoundSlo_HT1_1500_L1__MybestKaggle.csv', header=['prediction'])
